In [74]:
from mountaintools import client as mt
from spikeforest import SFMdaRecordingExtractor, SFMdaSortingExtractor
import spikeforestwidgets as SFW
import vdomr as vd
import numpy as np
import pandas as pd
vd.config_jupyter()

# IMPORTANT - since this notebook is checked in to the repo... do not include image outputs.
# clear outputs before committing.

# This is a work in progress

# An example command for James:
# > ./assemble_website_data.py --output_ids hybrid_janelia_irc,paired_kampff_irc --dest_key_path output.json
# then put the output.json in the path below.

JSProxyWidget(status='Not yet rendered')

In [105]:
# grab this URL from the website archive, or create your own
# all the paths can also point to local files
A = mt.loadObject(path='sha1://d0eb11774305a926e75ad232e4a6b4a54ffed4b2/analysis.json')

In [106]:
def find_study_analysis_result(A, study_name):
    for x in A['StudyAnalysisResults']:
        if x['studyName'] == study_name:
            return x

def compute_avg_accuracy(sar, sorting_result, snr_threshold):
    snrs = sar['trueSnrs']
    x = sorting_result['accuracies']
    x_to_use = [x[i] for i in range(len(x)) if x[i] is not None and snrs[i] is not None and snrs[i] >= snr_threshold]
    if x_to_use:
        return np.mean(x_to_use)
    return 0
        
snr_threshold = 8

sorters = A['Sorters']
for sset in A['StudySets']:
    display(vd.h5(sset['name']))
    table_rows = []
    for study in sset['studies']:
        sar = find_study_analysis_result(A, study['name'])
        row = dict(
            study=study['name']
        )
        for sr in sar['sortingResults']:
            sorter_name = sr['sorterName']
            val = compute_avg_accuracy(sar, sr, snr_threshold=snr_threshold)
            row[sorter_name] = val
        table_rows.append(row)
    df = pd.DataFrame(table_rows)
    df = df[['study'] + [sorter['name'] for sorter in sorters]].round(2)
    display(df)

,study,HerdingSpikes2,IronClust,JRClust,KiloSort,KiloSort2,Klusta,MountainSort4,SpykingCircus,Tridesclous,Waveclus
0,paired_boyden32c,0,0.6,0.59,0.68,0.73,0.36,0.43,0.5,0.19,0


,study,HerdingSpikes2,IronClust,JRClust,KiloSort,KiloSort2,Klusta,MountainSort4,SpykingCircus,Tridesclous,Waveclus
0,paired_crcns,0,0.73,0.7,0.56,0.68,0.77,0.83,0.75,0.41,0


,study,HerdingSpikes2,IronClust,JRClust,KiloSort,KiloSort2,Klusta,MountainSort4,SpykingCircus,Tridesclous,Waveclus
0,paired_mea64c,0.75,0.86,0.86,0.8,0.82,0.67,0.81,0.76,0.79,0


,study,HerdingSpikes2,IronClust,JRClust,KiloSort,KiloSort2,Klusta,MountainSort4,SpykingCircus,Tridesclous,Waveclus
0,paired_kampff,0.84,0.93,0.94,0.81,0.97,0.38,0.82,0.73,0.58,0


,study,HerdingSpikes2,IronClust,JRClust,KiloSort,KiloSort2,Klusta,MountainSort4,SpykingCircus,Tridesclous,Waveclus
0,paired_monotrode_boyden32c,0,0.18,0,0,0,0.38,0.38,0.28,0.17,0.29
1,paired_monotrode_kampff,0,0.10,0,0,0,0.49,0.51,0.34,0.24,0.36
2,paired_monotrode_crcns,0,0.35,0,0,0,0.28,0.35,0.64,0.21,0.52
3,paired_monotrode_mea64c,0,0.00,0,0,0,0.00,0.00,0.00,0.00,0.00


...

,study,HerdingSpikes2,IronClust,JRClust,KiloSort,KiloSort2,Klusta,MountainSort4,SpykingCircus,Tridesclous,Waveclus
0,manual_tetrode_600s,0,0.43,0.31,0.41,0.44,0.55,0.62,0.31,0.05,0
1,manual_tetrode_1200s,0,0.38,0.33,0.47,0.46,0.61,0.64,0.18,0.05,0
2,manual_tetrode_2400s,0,0.40,0.23,0.51,0.46,0.62,0.65,0.16,0.02,0


,study,HerdingSpikes2,IronClust,JRClust,KiloSort,KiloSort2,Klusta,MountainSort4,SpykingCircus,Tridesclous,Waveclus
0,synth_mearec_neuronexus_noise10_K10_C32,0,0.99,0.98,0.99,0.97,0.83,0.90,0.79,0.95,0
1,synth_mearec_neuronexus_noise10_K20_C32,0,0.98,0.93,0.98,0.97,0.87,0.90,0.87,0.88,0
2,synth_mearec_neuronexus_noise10_K40_C32,0,0.94,0.88,0.91,0.92,0.86,0.88,0.85,0.78,0
3,synth_mearec_neuronexus_noise20_K10_C32,0,1.00,0.89,1.00,1.00,0.89,0.78,0.87,0.92,0
4,synth_mearec_neuronexus_noise20_K20_C32,0,0.99,0.96,0.93,0.97,0.83,0.94,0.83,0.99,0
5,synth_mearec_neuronexus_noise20_K40_C32,0,0.99,0.96,0.98,0.96,0.92,0.88,0.88,0.91,0


,study,HerdingSpikes2,IronClust,JRClust,KiloSort,KiloSort2,Klusta,MountainSort4,SpykingCircus,Tridesclous,Waveclus
0,synth_mearec_tetrode_noise10_K10_C4,0,0.98,0.59,0.88,0.00,0.75,0.96,0.93,0.61,0
1,synth_mearec_tetrode_noise10_K20_C4,0,0.95,0.57,0.85,0.78,0.82,0.89,0.81,0.43,0
2,synth_mearec_tetrode_noise20_K10_C4,0,0.97,0.91,1.00,0.00,0.92,1.00,0.91,0.99,0
3,synth_mearec_tetrode_noise20_K20_C4,0,0.96,0.96,0.93,0.00,0.90,0.99,0.90,0.92,0


,study,HerdingSpikes2,IronClust,JRClust,KiloSort,KiloSort2,Klusta,MountainSort4,SpykingCircus,Tridesclous,Waveclus
0,mea_c30,0.85,0.91,0,0.9,0.98,0.0,0.84,0.79,0.67,0


,study,HerdingSpikes2,IronClust,JRClust,KiloSort,KiloSort2,Klusta,MountainSort4,SpykingCircus,Tridesclous,Waveclus
0,hybrid_drift_siprobe,0.49,0.88,0.51,0.71,0.89,0.56,0.51,0.56,0.45,0
1,hybrid_static_siprobe,0.85,0.96,0.94,0.93,0.97,0.87,0.86,0.87,0.50,0
2,hybrid_drift_tetrode,0.31,0.79,0.58,0.69,0.71,0.65,0.58,0.68,0.28,0
3,hybrid_static_tetrode,0.34,0.92,0.76,0.89,0.89,0.88,0.83,0.77,0.45,0


In [107]:
# James: here is how to superimpose an updated analysis on top of an existing one:

from_website = mt.loadObject(path='sha1://d0eb11774305a926e75ad232e4a6b4a54ffed4b2/analysis.json')
update = mt.loadObject(path='key://pairio/spikeforest/test1.json')
sorter_names_in_update = [s['name'] for s in update['Sorters']]
for sr in from_website['SortingResults']:
    if sr['sorterName'] not in sorter_names_in_update:
        update['SortingResults'].append(sr)
for sar in update['StudyAnalysisResults']:
    sarW = find_study_analysis_result(from_website, sar['studyName'])
    for sr in sarW['sortingResults']:
        if sr['sorterName'] not in sorter_names_in_update:
            sar['sortingResults'].append(sr)
for sorter in from_website['Sorters']:
    if sorter['name'] not in sorter_names_in_update:
        update['Sorters'].append(sorter)
        
A=update